# Изучение и обучение RecSys (**CIMO**)

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
import time
import random
import networkx as nx
import torch
from collections import defaultdict

from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, pipeline
import torch
from io import BytesIO
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.decomposition import IncrementalPCA
import warnings
from tqdm import tqdm

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import HeteroConv, GATConv, Linear
from torch_geometric.utils import negative_sampling


warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

## Выбор моделей. Анализ требований.

Итак, мы переходим к основной части нашей работы, ключевому составляющей приложения - рекомендательной системы.

Ключевая особенность CIMO заключается в том, что мы хотим сделать не только соло режим подбора фильмов, но и парный, в котором наша основная задача - за минимальное количество предложенных рекомендаций привести двух пользователей к "мэтчу", то есть к тому, чтобы предложенные им рекомендации, которые понравились им по отдельности, совпали.

В первую очередь определимся с требованиями, которые мы имеем по отношению к потенциальным решениям:
1. Скорость обучения и обновления
2. Учёт вкусов двух пользователей одновременно
3. Учёт истории взаимодействия пользователей ранее
4. Гибкость системы вне зависимости от выбранного режима
5. Простота обучения и использования
6. Масштабируемость

Ключевые параметры на которые модель должны обращать внимание:
1. Свайпы влево/вправо
2. Открытие подробной карточки фильма
3. Время просмотра карточки
4. Характеристики фильма


#### Одиночный режим

 Здесь простой коллаборативной фильтрации может не хватить, потому что она работает только на взаимодействиях пользователей и фильмов, но не учитывает контентные признаки, а для нас это необходимо. Поэтому тут лучше подойдут модели, которые совмещают коллаборативную фильтрацию + контентный анализ + временные зависимости.

#### Парный режим

Для парного режима мы решили сделать следующее. Парная и индивидуальная рекоменадции будут чередоваться для того, чтобы обновлять и насыщать индивидуальные векторы пользователей, в следствие чего будет насыщаться и общий вектор.

Фильмы должны быть релевантны сразу для двух пользователей, следовательно:
* Классическая коллаборативная фильтрация не подходит, поскольку она предсказывает предпочтения только одного пользователя
* Нужно учитывать пересечение интересов двух пользователей
* Важно анализировать динамику совместных решений

#### Промежуточные выводы

Итак, несложно заметить, что оптимальные решения для двух режимов отличаются. Поэтому перед нами встает выбор: либо для каждого режима выбирать отдельное решение и работать с ним, либо создать унифицированное решение, которое будет одинаково качественно работать в обоих режимах.

Мы остановились на втором варианте - **создании унифицированных представлений пользователей**.

То есть вместо двух отдельных моделей мы будем создавать один embedding. Если в сессии один человек, то embedding = его предпочтения. Если же в сессии пара, то embadding = среднее +  корректировка на совместимость.


Таким образом, мы сможем построить уникальную универсальную систему, которая сможет подстраиваться под разные сценарии. Это прекрасный вариант для стартапа, которым и является наше приложение, решение является очень гибким и быстрым в обучении. Более того, данная система будет готова и к дальнейшему потенциальному масштабированию.

### Заметки

Поскольку в модель будет попадать история взаимодействия юзера или юзеров с приложением, то при первой регистрации как один из вариантов - сделать пробную одиночную сессию, чтобы данные после нее использовать для нормальной рабочей сессии в будущем + это будет хороший `onboarding` для юзера, чтобы привыкнуть к интерфейсу и понять, как взаимодействовать с сервисом.

## Создание эмбеддингов для фильмов


In [ ]:
# !gdown https://drive.google.com/uc?id=1V5IdRQy4WCe4euRwGKwjKRunTKn4FhU7 -O "movie_roles.pkl"
# !gdown https://drive.google.com/uc?id=1MPnlqCXQ-I_cXjo0qiDxKwDrSUyjCdix -O "movies.pkl"
# !gdown https://drive.google.com/uc?id=10LUiHvA8ytso_ROVtW23AdBi-dXGsRmj -O "people.pkl"

In [ ]:
common_url = "/content/drive/MyDrive/processed_dataset"

movies_df = pd.read_pickle(f"{common_url}/movies.pkl")
people_df = pd.read_pickle(f"{common_url}/people.pkl")
movie_roles_df = pd.read_pickle(f"{common_url}/movie_roles.pkl")

In [ ]:
movies_df.sample()

,type,name,release_year,description,rating_kp,rating_imdb,runtime,age_rating,poster_url,genres,countries
9875,MOVIE,Гранд Отель,1932,Обычный день роскошного Берлинского гранд-отел...,7.2,7.3,112.0,18.0,/10703859/b876a4fd-cd8e-46fe-b55d-ec9517e5d58e...,"[драма, мелодрама]",[США]


In [ ]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99188 entries, 0 to 99187
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   type          99188 non-null  object 
 1   name          99188 non-null  object 
 2   release_year  99188 non-null  int64  
 3   description   99188 non-null  object 
 4   rating_kp     58572 non-null  float64
 5   rating_imdb   95420 non-null  float64
 6   runtime       95371 non-null  float64
 7   age_rating    42281 non-null  float64
 8   poster_url    99188 non-null  object 
 9   genres        98886 non-null  object 
 10  countries     99102 non-null  object 
dtypes: float64(4), int64(1), object(6)
memory usage: 8.3+ MB


### Попытка № 1

Будем использовать то, что имеем в нашем датасете.
- `Текстовый вектор` (описание + актёры)
- `Мета-вектор` (год выпуска, рейтинги, продолжительность, возрастное ограничение, жанры, страны, тип)

После этого, полученнные векторы нужно сконкатенировать и желательно преобразовать в другой вектор, чтобы снизить размерность и связать признаки.

#### Текстовые эмбеддинги

Сначала подготовим `текстовый вектор`

In [ ]:
import re

def prepare_text(data):
    people_str = data['people']
    description_str = data['description']
    people_info = f"В фильме принимали участие: {people_str}." if people_str != "Нет данных" else ""
    return re.sub(r'\s+', ' ', f"{description_str}. {people_info}").strip()


TOP_PEOPLE = 10
df = movie_roles_df.merge(people_df, left_on="person_id", right_index=True)
df = df.groupby("movie_id")["name"].apply(lambda x: ", ".join(x[:TOP_PEOPLE])).reset_index()
df = movies_df.merge(df, left_index=True, right_on="movie_id", how="left")
df.rename(columns={"name_y": "people"}, inplace=True)
df["people"].fillna("Нет данных", inplace=True)
df["merged"] = df.apply(prepare_text, axis=1)
df.rename(columns={"merged": "full_description"}, inplace=True)
df.reset_index(inplace=True)
df.sample()


,type,name_x,release_year,description,rating_kp,rating_imdb,runtime,age_rating,poster_url,genres,countries,movie_id,people,full_description
67269.0,CARTOON,Принцесса и гоблин,1991,"Король отправился в поход, оставив свою дочь п...",7.8,6.7,82.0,0.0,/1599028/1590f29a-f219-47d0-be63-f6b8090bd840/...,"[мультфильм, мюзикл, фэнтези, комедия, приключ...","[Великобритания, Венгрия, Япония, США, Дания]",67315,"Йожеф Гемеш, Джосс Экленд, Клэр Блум, Рой Кинн...","Король отправился в поход, оставив свою дочь п..."


In [ ]:
text_data = df[["full_description"]]
text_data.sample().to_numpy()

array([['Четырнадцатилетний Лукас больше интересуется тайнами науки и поведением насекомых, чем футболом и вечеринками с девушками. Но вот во время летних каникул он влюбляется в симпатичную шестнадцатилетнюю девчонку из группы поддержки.. В фильме принимали участие: Дэвид Зельцер, Кори Хэйм, Керри Грин, Чарли Шин, Кортни Торн-Смит, Вайнона Райдер, Том Ходжис, Чиро Поппити, Гай Бойд, Джереми Пивен.']],
      dtype=object)

Загрузим модели для обработки текстовых векторов. Будем использовать `RuBert` для работы с текстами на русском языке

In [ ]:
rubert_tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
rubert_model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")
rubert_model = rubert_model.to(device)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts.iloc[idx]["full_description"]

def text_collate_fn(batch):
    return batch


BATCH_SIZE = 256
text_dataset = TextDataset(text_data)
text_dataloader = DataLoader(text_dataset, batch_size=BATCH_SIZE, collate_fn=text_collate_fn, shuffle=False)

In [ ]:
def get_text_embeddings(texts):
    max_length = 512
    inputs = rubert_tokenizer(texts, return_tensors="pt", padding=True,
                              truncation=True, max_length=max_length).to(device)
    with torch.no_grad():
        outputs = rubert_model(**inputs)

    embeddings = outputs.last_hidden_state[:, 0, :].cpu()

    del inputs, outputs
    torch.cuda.empty_cache()
    return embeddings


#### Эмбеддинг мета-данных

Теперь займёмся `мета-вектором` и подготовим все признаки, применив **OneHotEncoder** и **бинаризацию**

In [ ]:
unique_countries, unique_genres = set(), set()

for countries in movies_df["countries"]:
    if isinstance(countries, list):
        unique_countries.update(countries)

for genres in movies_df["genres"]:
    if isinstance(genres, list):
        unique_genres.update(genres)

print(f"Кол-во уникальных стран: {len(unique_countries)}")
print(f"Кол-во уникальных жанров: {len(unique_genres)}")
print(f"Кол-во уникальных возрастных ограничений: {len(movies_df['age_rating'].unique())}")
print(f"Кол-во уникальных типов: {len(movies_df['type'].unique())}")

Кол-во уникальных стран: 211
Кол-во уникальных жанров: 25
Кол-во уникальных возрастных ограничений: 6
Кол-во уникальных типов: 3


In [ ]:
meta_data = movies_df[["countries", "genres", "age_rating", "type",
                       "release_year", "rating_kp", "rating_imdb", "runtime"]]
meta_data.sample()

,countries,genres,age_rating,type,release_year,rating_kp,rating_imdb,runtime
30127,[США],[драма],NaN,MOVIE,2007,NaN,6.7,NaN


In [ ]:
# Работа со столбцом `type`

ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(meta_data[['type']])
ohe_type = ohe.transform(meta_data[['type']]).toarray()

ohe_type_df = pd.DataFrame(ohe_type, columns=ohe.get_feature_names_out())
ohe_type_df.sample()

,type_ANIME,type_CARTOON,type_MOVIE
53806,0.0,0.0,1.0


In [ ]:
# Работа со столбцом `release_year`

min_year, max_year = meta_data['release_year'].min() // 10 * 10, meta_data['release_year'].max() // 10 * 10 + 10

bins = np.arange(min_year, max_year + 10, 10)
labels = [f"{i}-{i + 9}" for i in bins[:-1]]
bin_release_year = pd.cut(meta_data['release_year'], bins=bins, labels=labels, right=False)
ohe_release_year_df = pd.get_dummies(bin_release_year, dtype=int)
ohe_release_year_df.sample()


,1900-1909,1910-1919,1920-1929,1930-1939,1940-1949,1950-1959,1960-1969,1970-1979,1980-1989,1990-1999,2000-2009,2010-2019,2020-2029
67914,0,0,0,0,0,0,0,1,0,0,0,0,0


In [ ]:
# Рпбота со столбцом `age_rating`

age_rating_df = meta_data['age_rating'].apply(lambda x: -1 if pd.isna(x) else int(x)) # Или можно 0, или средним
age_rating_df.sample()

,age_rating
49698,-1


In [ ]:
# Работа со столбцами `rating_kp` & `rating_imdb`

def combine_ratings(row):
    imdb, kp = row['rating_imdb'], row['rating_kp']

    if not pd.isna(imdb) and not pd.isna(kp):
        return (imdb + kp) / 2
    elif not pd.isna(imdb):
        return imdb
    elif not pd.isna(kp):
        return kp
    else:
        return 0

rating_df = pd.DataFrame(meta_data.apply(combine_ratings, axis=1), columns=['rating'])
rating_df.sample()

,rating
7249,6.45


In [ ]:
# Работа со столбцом `runtime`

def runtime_category(runtime):
    if pd.isna(runtime): return 'unknown'
    elif runtime < 60: return 'short'
    elif runtime < 120: return 'medium'
    elif runtime < 180: return 'long'
    else: return 'epic'

runtime_df = pd.DataFrame(meta_data['runtime'].apply(runtime_category), columns=['runtime'])
ohe_runtime_df = pd.get_dummies(runtime_df, dtype=int)
ohe_runtime_df.sample()

,runtime_epic,runtime_long,runtime_medium,runtime_short,runtime_unknown
4002,0,0,1,0,0


In [ ]:
# Работа со столбцом `genres`

meta_data['genres'] = meta_data['genres'].apply(lambda x: x if isinstance(x, list) else [])
meta_data['genres'] = meta_data['genres'].apply(lambda x: x if x else ['неизвестный'])

mlb = MultiLabelBinarizer()
ohe_genres = mlb.fit_transform(meta_data['genres'])

ohe_genres_df = pd.DataFrame(ohe_genres, columns=mlb.classes_)
ohe_genres_df.sample()

,аниме,биография,боевик,вестерн,военный,детектив,детский,документальный,драма,история,...,неизвестный,приключения,реальное ТВ,семейный,спорт,триллер,ужасы,фантастика,фильм-нуар,фэнтези
64761,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Работа со столбцом `countries`

TOP_COUNTRIES = 20
top_countries = meta_data['countries'].explode().value_counts().head(TOP_COUNTRIES).index
meta_data['countries'] = meta_data['countries'].apply(lambda x: x if isinstance(x, list) else [])
meta_data['countries'] = meta_data['countries'].apply(lambda x: [c for c in x if c in top_countries])
meta_data['countries'] = meta_data['countries'].apply(lambda x: x if x else ['Другие'])

mlb = MultiLabelBinarizer()
ohe_countries = mlb.fit_transform(meta_data['countries'])

ohe_countries_df = pd.DataFrame(ohe_countries, columns=mlb.classes_)
ohe_countries_df.sample()

,Австралия,Бельгия,Великобритания,Германия,Германия (ФРГ),Гонконг,Другие,Индия,Испания,Италия,...,Китай,Корея Южная,Мексика,Польша,Россия,СССР,США,Франция,Швеция,Япония
49109,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


Сконкатенируем обработанные признаки в один датафрейм

In [ ]:
meta_data = pd.concat([ohe_type_df, ohe_release_year_df, age_rating_df, rating_df,
                       ohe_runtime_df, ohe_genres_df, ohe_countries_df], axis=1)
meta_data.sample()

,type_ANIME,type_CARTOON,type_MOVIE,1900-1909,1910-1919,1920-1929,1930-1939,1940-1949,1950-1959,1960-1969,...,Китай,Корея Южная,Мексика,Польша,Россия,СССР,США,Франция,Швеция,Япония
89489,0.0,0.0,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
meta_data.shape

(99188, 70)

In [ ]:
class MetaDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data.iloc[idx].to_numpy()


BATCH_SIZE = 256
meta_dataset = MetaDataset(meta_data)
meta_dataloader = DataLoader(meta_dataset, batch_size=BATCH_SIZE, shuffle=False)

#### Получение итоговых эмбеддингов

In [ ]:
n_components = 256
output_file = '/content/drive/MyDrive/movies_embeddings.csv'
ipca = IncrementalPCA(n_components=n_components)

# Первый проход — partial_fit
for batch_text, batch_meta in zip(text_dataloader, meta_dataloader):
    text_embeddings = get_text_embeddings(batch_text)
    meta_embeddings = torch.tensor(batch_meta)

    combined = torch.cat((text_embeddings, meta_embeddings), dim=1)
    ipca.partial_fit(combined.numpy())

# Второй проход — transform + запись в CSV
with open(output_file, 'w') as file:
    for batch_text, batch_meta in zip(text_dataloader, meta_dataloader):
        text_embeddings = get_text_embeddings(batch_text)
        meta_embeddings = torch.tensor(batch_meta)

        combined = torch.cat((text_embeddings, meta_embeddings), dim=1)
        reduced = ipca.transform(combined.numpy())
        pd.DataFrame(reduced).to_csv(file, header=False, index=False)


In [ ]:
movies_embeddings = pd.read_csv('/content/drive/MyDrive/movies_embeddings.csv', header=None)
movies_embeddings.shape

(99188, 256)

#### Тестирование

In [ ]:
movie_title_year_dict = {}

for index, row in movies_df.iterrows():
    movie_title = row['name']
    release_year = row['release_year']
    movie_title_year_dict[f"{movie_title} ({release_year})"] = index


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Функция для поиска схожести 2 фильмов по косинусному расстоянию
def calculate_movie_similarity(movie_name1, movie_name2):
    try:
        movie_id1 = movie_title_year_dict[movie_name1]
        movie_id2 = movie_title_year_dict[movie_name2]

        movie1_embedding = movies_embeddings.iloc[movie_id1].values.reshape(1, -1)
        movie2_embedding = movies_embeddings.iloc[movie_id2].values.reshape(1, -1)

        similarity = cosine_similarity(movie1_embedding, movie2_embedding)[0][0]
        print(similarity)

    except IndexError:
        print(f"One or both movie IDs not found in the DataFrame.")
        return -1


In [ ]:
# Функция для определения схожести n рандомных фильмов к текущему
def recommend_n_movies(movie_name, n_random=10):
    try:
        movie_index = movie_title_year_dict[movie_name]
        movie_embedding = movies_embeddings.iloc[movie_index].values.reshape(1, -1)

        similarities = []
        random_movies = random.sample(range(len(movies_embeddings)), n_random)

        for movie_id in random_movies:
            random_movie_embedding = movies_embeddings.iloc[movie_id].values.reshape(1, -1)
            similarity = cosine_similarity(movie_embedding, random_movie_embedding)[0][0]
            similarities.append((movie_id, similarity))

        similarities.sort(key=lambda x: x[1], reverse=True)
        for movie_id, similarity in similarities:
            movie_title = movies_df.iloc[movie_id]['name']
            release_year = movies_df.iloc[movie_id]['release_year']
            print(f"{movie_title} ({release_year}): {similarity}")
    except KeyError:
        print(f"Movie '{movie_name}' not found in the database.")


In [ ]:
calculate_movie_similarity("Интерстеллар (2014)", "Контакт (1997)")

0.9040874484505829


In [ ]:
calculate_movie_similarity("Человек-паук: Нет пути домой (2021)", "Человек-паук (2002)")

0.9255667254970432


In [ ]:
calculate_movie_similarity("Крик (1996)", "Челюсти (1975)")

0.9436187366869757


In [ ]:
calculate_movie_similarity("Интерстеллар (2014)", "Зверополис (2016)")

0.2271824043952178


In [ ]:
calculate_movie_similarity("Дурак (2014)", "Большой куш (2000)")

-0.8049784722072163


In [ ]:
calculate_movie_similarity("Крик (1996)", "Титаник (1997)")

# Плохой результат

0.8446250002390546


### Попытка № 2

Будем работать с самым важным признаком - `description`. Как мы увидели из прошлой попытки, описание, которые мы имеем, не содержит много информации. Оно не затрагивает сильно сюжет, а выступает в роли "затравки" для привлечения внимания пользователей к данному фильму. В связи с этим приходит мысль о том, где найти описания фильмов побольше. Первое, что приходит на ум, - попросить `LLM модель` сгенерировать это насыщенное описание (безусловно, с проверкой). Однако это весьма трудозатратная задача сгенерировать ~100к описаний даже на 250-300 слов. В качестве альтернативы попробуем использовать `Википедию` - другой большой ресурс информации.

In [ ]:
!nvidia-smi

Tue Apr 15 15:13:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=55d5ac4461300fa0cd06d8e01c2e7cf3fda0fdb7e6e77b31f95b4bc3dfa525f0
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [ ]:
import wikipedia

def extract_plot_section(content):
    pattern = r'==\s*(Сюжет|Содержание)\s*==(.+?)(==|$)'
    match = re.search(pattern, content, re.DOTALL | re.IGNORECASE)

    if match:
        return match.group(2).strip()
    else:
        return None

wikipedia.set_lang("ru")
title = "Титаник (1997)"
try:
    summary = wikipedia.page(title)
    print(extract_plot_section(summary.content))
except wikipedia.exceptions.DisambiguationError as e:
    print(f"Нужно уточнение. Возможные варианты: {e.options}")
except wikipedia.exceptions.PageError:
    print("Страница не найдена")


В 1996 году охотник за сокровищами Брок Лаветт и его команда на научно-исследовательском судне «Академик Мстислав Келдыш» на глубоководных батискафах «Мир-1» и «Мир-2» погружаются на дно Атлантического океана, где покоятся обломки «Титаника», затонувшего 15 апреля 1912 года. Они обнаруживают сейф, в котором, по предположению Лаветта, должен находиться кулон с голубым бриллиантом, известным как «Сердце океана». В начале XX века Натан Хокли, по прозвищу «Питтсбургский Ротшильд», приобрёл его для своего сына-промышленника Каледона «Кэла» Хокли, а тот подарил бриллиант в качестве свадебного подарка своей невесте — англичанке Розе Дьюитт-Бьюкейтер, вместе с которой плыл на «Титанике» в США, где они должны были пожениться. По некоторым данным известно, что Каледон спасся, Роза погибла, а драгоценность должна находиться в сейфе на дне океана. Но, к разочарованию исследователей, в сейфе оказываются только размокшие бумаги, среди которых хорошо сохранившийся портрет, датированный 14 апреля 1912

In [ ]:
count = 0
summaries = {}

def check_wikipedia_desc(title, year):
    global count
    page_name = f"{title} ({year})"
    try:
        summary = wikipedia.page(page_name)
        summaries[page_name] = summary.content
        count += 1
        return True
    except wikipedia.exceptions.DisambiguationError:
        return False
    except wikipedia.exceptions.PageError:
        return False


In [ ]:
BATCH_SIZE = 250
DELAY_BETWEEN_REQUESTS = 0.5
DELAY_BETWEEN_BATCHES = 3

start_idx, end_idx = 0, len(movies_df)

for batch_start in range(start_idx, end_idx, BATCH_SIZE):
    batch_end = min(batch_start + BATCH_SIZE, end_idx)

    for i in range(batch_start, batch_end):
        title = movies_df.loc[i, 'name']
        year = movies_df.loc[i, 'release_year']

        check_wikipedia_desc(title, year)
        time.sleep(DELAY_BETWEEN_REQUESTS)

    print(f"Батч {batch_start}-{batch_end} завершён. Пауза...\n")
    time.sleep(DELAY_BETWEEN_BATCHES)

# Очень долго + не на все фильмы есть статья в Википедии,
# так как датасет содержит достаточное кол-во непопулярных фильмов
# или данная статья называется по-другому =(

### Попытка №3


Попробуем получить эмбеддинги фильмов через графовые нейросети `GNN`. Путём соединения фильмов как узлов через ребра в виде различных полей, характеризующих конкретный фильм, получим мультирёбра и обширный граф. Его и нужно будет потом использовать для обучения модели.

#### Создание графа nx.Graph

In [ ]:
def prepare_text(data):
    people_str = data['people']
    description_str = data['description']
    people_info = f"В фильме принимали участие: {people_str}." if people_str != "Нет данных" else ""
    return re.sub(r'\s+', ' ', f"{description_str}. {people_info}").strip()


TOP_PEOPLE = 10
df = movie_roles_df.merge(people_df, left_on="person_id", right_index=True)
df = df.groupby("movie_id")["name"].apply(lambda x: ", ".join(x[:TOP_PEOPLE])).reset_index()
df = movies_df.merge(df, left_index=True, right_on="movie_id", how="left")
df.rename(columns={"name_y": "people", "name_x": "name"}, inplace=True)
df["people"].fillna("Нет данных", inplace=True)
df["merged"] = df.apply(prepare_text, axis=1)
df.rename(columns={"merged": "full_description"}, inplace=True)
df.reset_index(inplace=True, drop=True)
df.sample()

,type,name,release_year,description,rating_kp,rating_imdb,runtime,age_rating,poster_url,genres,countries,movie_id,people,full_description
96980,MOVIE,Семь вуалей,2023,Молодому театральному режиссеру Джанин поручаю...,NaN,6.6,107.0,18.0,/10953618/80d88800-26e1-4a2e-a482-697fcd05a653...,[драма],"[Канада, Финляндия, США]",96980,"Бай Хун, Мартон Аг, Бен Дискин, Чейз Суи Уонде...",Молодому театральному режиссеру Джанин поручаю...


Создадим пустой `мультиграф`

In [ ]:
G = nx.Graph()

In [ ]:
for idx, row in tqdm(df.iterrows(), total=len(df)):
    imdb_rating = row['rating_imdb']
    kp_rating = row['rating_kp']
    weight = None

    if pd.notnull(imdb_rating) and pd.notnull(kp_rating):
        weight = (imdb_rating + kp_rating) / 2
    elif pd.notnull(imdb_rating):
        weight = imdb_rating
    else:
        weight = kp_rating
    G.add_node(f"film_{idx + 1}", type='film', name=row['name'], year=row['release_year'], weight=weight)

print(f"\nВсего узлов-фильмов: {G.number_of_nodes()}")

100%|██████████| 99188/99188 [00:12<00:00, 8127.15it/s] 


Всего узлов-фильмов: 99188


In [ ]:
for idx, row in tqdm(df.iterrows(), total=len(df)):
    film_id = f"film_{idx + 1}"

    # Тип
    if pd.notnull(row['type']):
        type_node = f"type_{row['type']}"
        G.add_node(type_node, type='type')
        G.add_edge(film_id, type_node, relation='rtype', weight=0.0)

    # Жанры
    genres = row['genres']
    if isinstance(genres, list):
        for genre in genres:
            genre_node = f"genre_{genre}"
            G.add_node(genre_node, type='genre')
            G.add_edge(film_id, genre_node, relation='rgenre', weight=0.0)

    # Страны
    countries = row['countries']
    if isinstance(countries, list):
        for country in countries:
            country_node = f"country_{country}"
            G.add_node(country_node, type='country')
            G.add_edge(film_id, country_node, relation='rcountry', weight=0.0)

    # Возрастной рейтинг
    if pd.notnull(row['age_rating']):
        age_bin = f"age_{int(row['age_rating'])}"
        G.add_node(age_bin, type='age')
        G.add_edge(film_id, age_bin, relation='rage', weight=0.0)

    # Год релиза (по десятилетиям)
    if pd.notnull(row['release_year']):
        decade = int(row['release_year']) // 10 * 10
        year_bin = f"decade_{decade}s"
        G.add_node(year_bin, type='decade')
        G.add_edge(film_id, year_bin, relation='rdecade', weight=0.0)

100%|██████████| 99188/99188 [00:12<00:00, 8107.40it/s] 


Проведем анализ графа для проверки узлов и ребер.

In [ ]:
num_nodes = len(G.nodes())
num_edges = len(G.edges())

print(f"Число узлов: {num_nodes}")
print(f"Число рёбер: {num_edges}")

Число узлов: 99445
Число рёбер: 594288


In [ ]:
node_types = {'film': 0, 'type': 0, 'genre': 0, 'country': 0, 'age': 0, 'decade': 0}

for node in G.nodes():
    node_type = G.nodes[node].get('type', None)
    if node_type in node_types:
        node_types[node_type] += 1

print("Типы узлов в графе:", node_types)

Типы узлов в графе: {'film': 99188, 'type': 3, 'genre': 25, 'country': 211, 'age': 5, 'decade': 13}


In [ ]:
nx.is_connected(G)

True

In [ ]:
degree_centrality = nx.degree_centrality(G)

print("Топ 10 узлов по степени центральности:")
top_10_degree = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:10]
print(*top_10_degree, sep='\n')


Топ 10 узлов по степени центральности:
('type_MOVIE', 0.9371505570974619)
('genre_драма', 0.526296206910422)
('country_США', 0.34061381279916336)
('genre_комедия', 0.2873275411286754)
('decade_2010s', 0.2728671413056595)
('decade_2000s', 0.19652266602308838)
('genre_мелодрама', 0.18969470254615664)
('age_18', 0.17286110775914082)
('genre_триллер', 0.16006998913961626)
('genre_боевик', 0.13724307147741444)


Теперь через BERT построим эмбеддинги описаний и далее добавим их в граф, давая связям между фильмами тот вес, который соответствует косинусному расстоянию между веторами фильмов.

In [ ]:
model_name = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.eval()

# Тексты описаний
descriptions = df['full_description'].tolist()

# Функция для одного батча
def embed_batch(text_batch):
    tokens = tokenizer(text_batch, padding=True, truncation=True, return_tensors="pt", max_length=512)
    tokens = {k: v.to(device) for k, v in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
    embeddings = outputs.last_hidden_state[:, 0, :]
    return embeddings.cpu().numpy()

BATCH_SIZE = 64
all_embeddings = []

for i in tqdm(range(0, len(descriptions), BATCH_SIZE), desc="Embedding descriptions"):
    batch = descriptions[i : i + BATCH_SIZE]
    all_embeddings.append(embed_batch(batch))

description_embeddings = np.vstack(all_embeddings)

In [ ]:
dim = description_embeddings.shape[-1]
dim

768

Ищем 20 ближайших векторов к каждому фильму.

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 69.2 MB/s eta 0:00:00


In [ ]:
import faiss

faiss.normalize_L2(description_embeddings)

index = faiss.IndexFlatIP(dim)
index.add(description_embeddings)

# D — косинусные расстояния, I — индексы ближайших фильмов
D, I = index.search(description_embeddings, 25)

In [ ]:
film_similarities = []
id_map = [f"film_{i + 1}" for i in range(len(description_embeddings))]

for idx, (neighbors, scores) in enumerate(zip(I, D)):
    film_a = id_map[idx]
    for neighbor_idx, score in zip(neighbors[1:], scores[1:]):
        film_b = id_map[neighbor_idx]
        if score > 0.85:
            film_similarities.append((film_a, film_b, float(score)))

similarity_df = pd.DataFrame(film_similarities, columns=['film_a', 'film_b', 'similarity'])

similarity_df.to_csv('film_similarity.csv', index=False)

In [ ]:
!cp film_similarity.csv /content/drive/MyDrive/film_similarity.csv

In [ ]:
file_path = "/content/drive/MyDrive/film_similarity.csv"
similarity_df = pd.read_csv(file_path)

In [ ]:
similarity_df.shape

(2380404, 3)

Добавляем ребра по схожести описания

In [ ]:
for _, row in tqdm(similarity_df.iterrows(), total=len(similarity_df)):
    film_idx_a = row['film_a']
    film_idx_b = row['film_b']
    similarity = float(row['similarity'])
    G.add_edge(film_idx_a, film_idx_b, relation='rsimilar_by_description', weight=similarity)

100%|██████████| 2380404/2380404 [02:45<00:00, 14351.56it/s]


In [ ]:
print(f"Число узлов: {len(G.nodes())}")
print(f"Число рёбер: {len(G.edges())}")

Число узлов: 99445
Число рёбер: 2755605


In [ ]:
all('weight' in data for u, v, data in G.edges(data=True))

True

In [ ]:
all('relation' in data for u, v, data in G.edges(data=True))

True

In [ ]:
max_weight = max(data['weight'] for _, _, data in G.edges(data=True))
min_weight = min(data['weight'] for _, _, data in G.edges(data=True))

print(f"Максимальный вес: {max_weight}")
print(f"Минимальный вес: {min_weight}")

Максимальный вес: 0.9991852045059204
Минимальный вес: 0.0


#### Подготовка графа G к обучению


In [ ]:
data = HeteroData()

# Отображения: имя узла -> индекс
node_maps = defaultdict(dict)
node_features = defaultdict(list)
# Счётчики узлов по типам
node_type_counts = defaultdict(int)

In [ ]:
for node, attr in G.nodes(data=True):
    ntype = attr.get("type")
    if ntype == "film":
        idx = node_type_counts[ntype]
        node_maps[ntype][node] = idx
        weight = attr.get("weight")
        year = attr.get("year") / 2025
        node_features[ntype].append([weight, year])
        node_type_counts[ntype] += 1
    else:
        idx = node_type_counts[ntype]
        node_maps[ntype][node] = idx
        node_type_counts[ntype] += 1


In [ ]:
data['film'].x = torch.tensor(node_features['film'], dtype=torch.float)

for ntype in node_maps:
    if ntype != 'film':
        data[ntype].num_nodes = len(node_maps[ntype])


In [ ]:
# Для хранения рёбер по типам
edge_indices = defaultdict(list)
edge_weights = defaultdict(list)

for u, v, attr in G.edges(data=True):
    rel = attr.get('relation')

    src_type = G.nodes[u]['type']
    dst_type = G.nodes[v]['type']

    # В случае фильма нужно соблюдать направление film -> атрибут
    if src_type == 'film' and dst_type != 'film':
        src, dst = u, v
    elif dst_type == 'film' and src_type != 'film':
        src, dst = v, u
    elif src_type == 'film' and dst_type == 'film':
        src, dst = u, v
    else:
        continue

    src_index = node_maps[src_type][src]
    dst_index = node_maps[dst_type][dst]

    edge_type = (src_type, rel, dst_type)
    edge_indices[edge_type].append((src_index, dst_index))
    edge_weights[edge_type].append(attr.get('weight', 0.0))

In [ ]:
for edge_type, edge_list in edge_indices.items():
    src_type, rel, dst_type = edge_type
    edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_weights[edge_type], dtype=torch.float).view(-1, 1)

    data[(src_type, rel, dst_type)].edge_index = edge_index
    data[(src_type, rel, dst_type)].edge_attr = edge_attr


In [ ]:
data = data.to(device)
data

HeteroData(
  film={ x=[99188, 2] },
  type={ num_nodes=3 },
  genre={ num_nodes=25 },
  country={ num_nodes=211 },
  age={ num_nodes=5 },
  decade={ num_nodes=13 },
  (film, rtype, type)={
    edge_index=[2, 99188],
    edge_attr=[99188, 1],
  },
  (film, rgenre, genre)={
    edge_index=[2, 223506],
    edge_attr=[223506, 1],
  },
  (film, rcountry, country)={
    edge_index=[2, 130125],
    edge_attr=[130125, 1],
  },
  (film, rage, age)={
    edge_index=[2, 42281],
    edge_attr=[42281, 1],
  },
  (film, rdecade, decade)={
    edge_index=[2, 99188],
    edge_attr=[99188, 1],
  },
  (film, rsimilar_by_description, film)={
    edge_index=[2, 2161317],
    edge_attr=[2161317, 1],
  }
)

#### Обучение графовой нейросети GNN

In [ ]:
class FilmGNN(nn.Module):
    def __init__(self, hidden_dim=128, out_dim=128):
        super().__init__()
        self.conv1 = HeteroConv({
            ('film', 'rtype', 'type'): GATConv((-1, -1), hidden_dim, heads=1, concat=False, add_self_loops=False),
            ('film', 'rgenre', 'genre'): GATConv((-1, -1), hidden_dim, heads=1, concat=False, add_self_loops=False),
            ('film', 'rcountry', 'country'): GATConv((-1, -1), hidden_dim, heads=1, concat=False, add_self_loops=False),
            ('film', 'rage', 'age'): GATConv((-1, -1), hidden_dim, heads=1, concat=False, add_self_loops=False),
            ('film', 'rdecade', 'decade'): GATConv((-1, -1), hidden_dim, heads=1, concat=False, add_self_loops=False),
            ('film', 'rsimilar_by_description', 'film'): GATConv((-1, -1), hidden_dim, heads=1, concat=False),
        }, aggr='sum')

        self.lin = Linear(hidden_dim, out_dim)

    def forward(self, x_dict, edge_index_dict, edge_attr_dict):
        x_dict = self.conv1(x_dict, edge_index_dict, edge_attr_dict=edge_attr_dict)
        x_dict['film'] = self.lin(x_dict['film'])
        return x_dict


In [ ]:
model = FilmGNN(hidden_dim=128, out_dim=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

edge_index = data[('film', 'rsimilar_by_description', 'film')].edge_index
EPOCHS = 50

for epoch in range(EPOCHS):
    model.train()
    out_dict = model(data.x_dict, data.edge_index_dict, data.edge_attr_dict)
    film_emb = out_dict['film']  # [num_films, 128]

    pos_src = film_emb[edge_index[0]]
    pos_dst = film_emb[edge_index[1]]
    pos_scores = (pos_src * pos_dst).sum(dim=1)
    pos_labels = torch.ones_like(pos_scores)

    neg_edge_index = negative_sampling(
        edge_index=edge_index,
        num_nodes=film_emb.size(0),
        num_neg_samples=edge_index.size(1),
        method='sparse'
    )
    neg_src = film_emb[neg_edge_index[0]]
    neg_dst = film_emb[neg_edge_index[1]]
    neg_scores = (neg_src * neg_dst).sum(dim=1)
    neg_labels = torch.zeros_like(neg_scores)

    scores = torch.cat([pos_scores, neg_scores])
    labels = torch.cat([pos_labels, neg_labels])
    loss = F.binary_cross_entropy_with_logits(scores, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 5 == 0:
        with torch.no_grad():
            print(f"Epoch {epoch:>2}/{EPOCHS}, Loss: {loss.item()}")

Epoch  0/50, Loss: 14.707913398742676
Epoch  5/50, Loss: 2.305377244949341
Epoch 10/50, Loss: 1.1480066776275635
Epoch 15/50, Loss: 1.122639775276184
Epoch 20/50, Loss: 0.9047917723655701
Epoch 25/50, Loss: 0.7986855506896973
Epoch 30/50, Loss: 0.755832850933075
Epoch 35/50, Loss: 0.7235291004180908
Epoch 40/50, Loss: 0.7057123780250549
Epoch 45/50, Loss: 0.6989659667015076


In [ ]:
model.eval()
movies_embeddings = model(data.x_dict, data.edge_index_dict, data.edge_attr_dict)['film']  # [99188, 128]

In [ ]:
movies_embeddings.shape

torch.Size([99188, 128])

#### Тестирование

In [ ]:
movie_title_year_dict = {}
movies_embeddings = movies_embeddings.cpu().detach().numpy()

for index, row in movies_df.iterrows():
    movie_title = row['name']
    release_year = row['release_year']
    movie_title_year_dict[f"{movie_title} ({release_year})"] = index

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_movie_similarity(movie_name1, movie_name2):
    try:
        movie_id1 = movie_title_year_dict[movie_name1]
        movie_id2 = movie_title_year_dict[movie_name2]

        movie1_embedding = movies_embeddings[movie_id1][np.newaxis, :]
        movie2_embedding = movies_embeddings[movie_id2][np.newaxis, :]

        similarity = cosine_similarity(movie1_embedding, movie2_embedding)[0][0]
        print(similarity)

    except IndexError:
        print(f"One or both movie IDs not found in the DataFrame.")
        return -1


In [ ]:
calculate_movie_similarity("Человек-паук: Нет пути домой (2021)", "Человек-паук (2002)")

0.918236


In [ ]:
calculate_movie_similarity("Крик (1996)", "Титаник (1997)")

# Плохой результат

0.9452101
